# 2 Clean Deduped Dataset

The `interim/all_combined_deduped.csv` dataset is deduped. Now want to do a basic clean up and formatting of the dataset.

## Setup

In [1]:
import pandas as pd

In [2]:
df = (
    pd.read_csv("data/interim/1_combined_deduped.csv", low_memory=False)
    .sort_values(by=["origin", "sample"])
    .drop("heat_units", axis=1)
)
wavelengths = df.filter(regex="^\d+", axis=1).columns.tolist()

## Look at data

In [3]:
df.groupby("origin").size()

origin
2022_ABS_AllMangoNIR                                80629
MarceloSeveralVarieties_2018-2020_absorbancedata     1766
NAnderson2020MendeleyMangoNIRData                   11691
dtype: int64

In [4]:
df.dtypes

origin        object
population    object
date          object
season         int64
region        object
               ...  
1188           int64
1191           int64
1194           int64
1197           int64
1200           int64
Length: 317, dtype: object

In [5]:
df

,origin,population,date,season,region,cultivar,type,temp,dm,instrument,...,1173,1176,1179,1182,1185,1188,1191,1194,1197,1200
3001,2022_ABS_AllMangoNIR,1,2015-09-27,1,NT,Caly,Hard Green,High,15.901383,15028,...,0,0,0,0,0,0,0,0,0,0
10274,2022_ABS_AllMangoNIR,9,2015-10-23,1,NT,KP,Hard Green,No,13.705404,15029,...,0,0,0,0,0,0,0,0,0,0
3002,2022_ABS_AllMangoNIR,1,2015-09-27,1,NT,Caly,Hard Green,High,14.508961,15028,...,0,0,0,0,0,0,0,0,0,0
3003,2022_ABS_AllMangoNIR,1,2015-09-27,1,NT,Caly,Hard Green,High,14.508961,15028,...,0,0,0,0,0,0,0,0,0,0
3004,2022_ABS_AllMangoNIR,1,2015-09-27,1,NT,Caly,Hard Green,High,14.098233,15028,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90694,NAnderson2020MendeleyMangoNIRData,80,2017-12-13,3,QLD,KP,Ripen,No,15.664130,15006,...,0,0,0,0,0,0,0,0,0,0
91057,NAnderson2020MendeleyMangoNIRData,80,2017-12-13,3,QLD,KP,Ripen,No,16.229150,15006,...,0,0,0,0,0,0,0,0,0,0
90456,NAnderson2020MendeleyMangoNIRData,80,2017-12-13,3,QLD,KP,Ripen,No,16.229150,15006,...,0,0,0,0,0,0,0,0,0,0
91440,NAnderson2020MendeleyMangoNIRData,80,2017-12-13,3,QLD,KP,Ripen,No,16.649160,15006,...,0,0,0,0,0,0,0,0,0,0


## Clean up data

Will do this column by column.

In [6]:
# rename origin
df.loc[df["origin"] == "NAnderson2020MendeleyMangoNIRData", "origin"] = "published"
df.loc[df["origin"] == "2022_ABS_AllMangoNIR", "origin"] = "nich"
df.loc[df["origin"] == "MarceloSeveralVarieties_2018-2020_absorbancedata", "origin"] = "marcelo"

In [7]:
# clean population name to ints
def process_population(value, season):
    if value == "AHF":
        return 200100 if season == 2018 else 200300
    elif value == "Grovesfarm":
        return 200200 if season == 2018 else 200400
    elif value == "HG1":
        return 200500
    elif value == "HG4":
        return 200600
    elif value == "Young Keitt":
        return 200700
    elif value == "Old Keitt":
        return 200800
    elif isinstance(value, int)  or value.isdigit():
        return int(value) * 100 + 100000
    elif value[-1].lower() in ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k']:
        letter_value = ord(value[-1].lower()) - ord('a')
        return int(value[:-1]) * 100 + 100000 + letter_value
    else:
        raise ValueError(value)
df["population"] = df.apply(lambda p: process_population(p["population"], p["season"]), axis=1)

In [8]:
process_population(127, 2019)

112700

In [9]:
# format date and create season year
df["date"] = pd.to_datetime(df["date"])
df.insert(
    loc=3,
    column="season_year",
    value=df["date"].map(lambda x: x.year if x.month > 4 else x.year-1)
)
df["season_year"] = df["season_year"].fillna(df["season"]).astype(int)

In [10]:
# rename cultivars
df.loc[df["cultivar"] == "HoneyGold", "cultivar"] = "HG"
df.loc[df["cultivar"] == "Calypso", "cultivar"] = "Caly"
df["cultivar"] = df["cultivar"].str.lower()

In [11]:
# rename the physiological type
df["type"] = df["type"].fillna("green")
df.loc[df["type"] == "Hard Green", "type"] = "green"
df.loc[df["type"] == "Ripen", "type"] = "ripen"


In [12]:
# clean up temperature values
df.insert(
    loc=9,
    column="temp_celsius",
    value= pd.to_numeric(df["temp"], errors='coerce')
)
df.loc[df["temp_celsius"].between(0, 15, inclusive="left"), "temp"] = "Low"
df.loc[df["temp_celsius"].between(15, 30, inclusive="left"), "temp"] = "Mid"
df.loc[df["temp_celsius"].between(30, 50, inclusive="left"), "temp"] = "High"
df.loc[((df["temp"] == "No") | (df["temp"].isnull())), "temp"] = "Undef"
df["temp"] = df["temp"].str.lower()

In [13]:
# label reference (dm) values
df.insert(
    loc=10,
    column="reference_no",
    value=(df.groupby(["population", "dm"], sort=True).ngroup() + 1)
)

In [14]:
# clean up bad reference values (dm)
df_bad_ref = df.query("dm == 0")
df = df.query("dm != 0")
df = df.query("not (population == 111202 and dm == 22.85305)")
df_bad_ref

,origin,population,date,season_year,season,region,cultivar,type,temp,temp_celsius,...,1173,1176,1179,1182,1185,1188,1191,1194,1197,1200
40140,nich,109700,2018-08-07,2018,4,NT,caly,ripen,undef,NaN,...,0,0,0,0,0,0,0,0,0,0
40141,nich,109700,2018-08-07,2018,4,NT,caly,ripen,undef,NaN,...,0,0,0,0,0,0,0,0,0,0
40220,nich,109700,2018-08-07,2018,4,NT,caly,ripen,undef,NaN,...,0,0,0,0,0,0,0,0,0,0
40221,nich,109700,2018-08-07,2018,4,NT,caly,ripen,undef,NaN,...,0,0,0,0,0,0,0,0,0,0
40300,nich,109700,2018-08-07,2018,4,NT,caly,ripen,undef,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41095,nich,109900,2018-08-07,2018,4,NT,caly,ripen,undef,NaN,...,0,0,0,0,0,0,0,0,0,0
41096,nich,109900,2018-08-07,2018,4,NT,caly,ripen,undef,NaN,...,0,0,0,0,0,0,0,0,0,0
41097,nich,109900,2018-08-07,2018,4,NT,caly,ripen,undef,NaN,...,0,0,0,0,0,0,0,0,0,0
41098,nich,109900,2018-08-07,2018,4,NT,caly,ripen,undef,NaN,...,0,0,0,0,0,0,0,0,0,0


In [119]:
# clean up bad absorbance values
for x in [x for x in wavelengths if int(x) > 1000]:
    df.loc[df[x].abs() > 10, x] = 0

In [120]:
# clean up instrument names
df.loc[df["instrument"]=="ENG4", "instrument"] = 30004
df.loc[df["instrument"]=="Ryan", "instrument"] = 30000
df["instrument"] = df["instrument"].astype(int)

C:\Users\12196478\AppData\Local\Temp\ipykernel_28492\2335806089.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["instrument"] = df["instrument"].astype(int)


In [121]:
# label spectra values
df.sort_values(by=["population", "reference_no", "instrument"], inplace=True)
df.insert(
    loc=13,
    column="spectra_no",
    value=(df.reset_index().index + 1)
)

C:\Users\12196478\AppData\Local\Temp\ipykernel_28492\1622216838.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by=["population", "reference_no", "instrument"], inplace=True)


## Output cleaned data

Will output both a csv and pickle file. The csv will be good for manual viewing but the pickle is preferred as the dtypes of each column are retained.

In [122]:
df_out = df[
    [
        "origin",
        "population",
        "date",
        "season_year",
        "region",
        "cultivar",
        "type",
        "temp",
        "reference_no",
        "dm",
        "instrument",
        "spectra_no"
    ]
    + wavelengths
].reset_index(drop=True)
df_out.rename(
    columns={
        "dm": "dry_matter",
        "season_year": "season",
        "type": "physio_stage"
    },
    inplace=True
)
df_out[wavelengths] = df_out[wavelengths].astype(float)
df_out

,origin,population,date,season,region,cultivar,physio_stage,temp,reference_no,dry_matter,...,1173,1176,1179,1182,1185,1188,1191,1194,1197,1200
0,published,100100,2015-09-27,2015,NT,caly,green,high,1,12.402643,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,published,100100,2015-09-27,2015,NT,caly,green,high,1,12.402643,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,published,100100,2015-09-27,2015,NT,caly,green,low,1,12.402643,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,published,100100,2015-09-27,2015,NT,caly,green,low,1,12.402643,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,nich,100100,2015-09-27,2015,NT,caly,green,high,1,12.402643,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93943,marcelo,200800,NaT,2021,QLD,keitt,green,undef,10676,22.502700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93944,marcelo,200800,NaT,2021,QLD,keitt,green,undef,10677,22.888390,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93945,marcelo,200800,NaT,2021,QLD,keitt,green,undef,10678,22.975740,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93946,marcelo,200800,NaT,2021,QLD,keitt,green,undef,10679,23.211150,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [123]:
df_out.dtypes

origin                object
population             int64
date          datetime64[ns]
season                 int32
region                object
                   ...      
1188                 float64
1191                 float64
1194                 float64
1197                 float64
1200                 float64
Length: 318, dtype: object

In [124]:
# output
df_out.to_pickle("data/interim/2_cleaned.pkl")
df_out.to_csv("data/prepared/mango_dmc_and_spectra_v2.csv", index=False)
df_out.to_pickle("data/prepared/mango_dmc_and_spectra_v2.pkl")